In [ ]:
!pip install -U transformers
!pip install datasets
!pip install -U accelerate

In [1]:
%load_ext autoreload
%autoreload 2

> this notebook will follow the tutorial in:
https://blog.gopenai.com/fine-tuning-dialogpt-medium-on-daily-dialog-dataset-a-step-by-step-guide-4eaecc1b9323

# make my own dataset

In [2]:
# https://huggingface.co/learn/nlp-course/chapter5/5
# https://huggingface.co/learn/nlp-course/chapter5/2

In [3]:
from datasets import load_dataset
from glob import glob

/home/peterdays/Documents/personal/Volupal/ElevaQ/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
glob('data/*.json')

['data/DL103_2008.json',
 'data/0288702916.json',
 'data/DL320_2002.json',
 'data/L65_2013.json',
 'data/dlr7_2016-m.json',
 'data/0331103315.json',
 'data/DLR4_2012_A.json']

we can also set the splits

`data_files = {"train": "json_example.json", "test": "json_example.json"}`
`dataset = load_dataset("json", data_files=data_files, field="data")`

In [5]:
dataset = load_dataset("json", data_files=glob('data/*.json'))

# train test split
dataset = dataset["train"].train_test_split(test_size=0.20)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'page'],
        num_rows: 84
    })
    test: Dataset({
        features: ['text', 'page'],
        num_rows: 22
    })
})

In [7]:
dataset["train"][0]

{'text': ['Diário da República, 1.ª série — N.º 12 — 17 de janeiro de 2012 Artigo 8.º Responsabilidade pelo projeto e pela execução A responsabilidade técnica pela demonstração do cum- primento das exigências decorrentes do presente diploma apenas pode ser assumida por técnicos que cumpram o disposto no artigo 97.º do Decreto Legislativo Regional n.º 16/2009/A, de 13 de outubro, considerando- se uma qualificação específica para o efeito a definir nos termos do n.º 2 daquele artigo',
  'Artigo 9.º Obrigação de conformidade Apenas podem ser instalados dispositivos e seus com- ponentes de segurança em relação aos quais tenha sido emitida uma declaração CE de conformidade e que osten- tem aposta, nos termos legais e regulamentares aplicáveis, a respetiva marcação CE de conformidade',
  'Artigo 10.º Competências das câmaras municipais 1 — Sem prejuízo das atribuições e competências le- galmente atribuídas ou delegadas a outras entidades, as câmaras municipais, no âmbito do presente diploma,

In [8]:
# Concatenate all utterances within a dialogue and map to 'dialog' key
def concatenate_paragraphs(example):
    example['page'] = " ".join(example['text'])
    return example

# dataset = dataset.map(concatenate_paragraphs)

> Note: not sure if this is really needed, but for simplicity will make a whole text per example
>
> The DialogPT is based on short context, this doesn't lead to good results! Will be trying without concatenating as below

In [9]:
def flatten_list_of_dict(batch):
    return {"page": [ex_string for ex_list in batch["text"] for ex_string in ex_list]}

dataset = dataset.map(flatten_list_of_dict, batched=True, remove_columns=["text"])

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 3803.42 examples/s]


In [10]:
# re-joining the words separated by "-"
def text_processing(example):
    example['page'] = example['page'].replace("- ", "")
    return example

dataset = dataset.map(text_processing)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:00<00:00, 30399.86 examples/s]


In [11]:
dataset["train"][:10]

{'page': ['Diário da República, 1.ª série — N.º 12 — 17 de janeiro de 2012 Artigo 8.º Responsabilidade pelo projeto e pela execução A responsabilidade técnica pela demonstração do cumprimento das exigências decorrentes do presente diploma apenas pode ser assumida por técnicos que cumpram o disposto no artigo 97.º do Decreto Legislativo Regional n.º 16/2009/A, de 13 de outubro, considerandose uma qualificação específica para o efeito a definir nos termos do n.º 2 daquele artigo',
  'Artigo 9.º Obrigação de conformidade Apenas podem ser instalados dispositivos e seus componentes de segurança em relação aos quais tenha sido emitida uma declaração CE de conformidade e que ostentem aposta, nos termos legais e regulamentares aplicáveis, a respetiva marcação CE de conformidade',
  'Artigo 10.º Competências das câmaras municipais 1 — Sem prejuízo das atribuições e competências legalmente atribuídas ou delegadas a outras entidades, as câmaras municipais, no âmbito do presente diploma, são compe

### DETOUR!!

In [12]:
import os

os.environ["LD_LIBRARY_PATH"]=""

In [15]:
# TODO: FIX TORCH VERSION  -> this one was not the one original!!
!pip3 install torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 50.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 8.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 14.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 24.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 33.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 33.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 22.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 23.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━

# DialogGPT 

## Encoding

In [11]:
import numpy as np
import tempfile
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')

In [12]:
# Encode the dataset
# https://huggingface.co/docs/transformers/en/pad_truncation
def encode(examples):
    encoded = tokenizer(examples['page'],
                        truncation=True, 
                        padding='max_length',
                        max_length=128
                       )
    encoded['labels'] = encoded['input_ids'][:]

    return encoded

encoded_dataset = dataset.map(encode, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████████████████| 375/375 [00:00<00:00, 2216.13 examples/s]


## Training

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=tempfile.mkdtemp(),   # output directory
    num_train_epochs=25,             # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,                # directory for storing logs
    fp16=True                        # use floating point 16 bit precision for training
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

## Evaluation

In [14]:
# Evaluate before fine-tuning
pre_eval_results = trainer.evaluate(encoded_dataset['test'])

In [15]:
# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset['test'].select(range(10)))

## Fine tuning

In [16]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,3.752400
1000,1.780600
1500,1.448100
2000,1.242200
2500,1.104800
3000,1.003900
3500,0.924200
4000,0.870500
4500,0.835400


TrainOutput(global_step=4625, training_loss=1.423659377639358, metrics={'train_runtime': 684.5815, 'train_samples_per_second': 53.938, 'train_steps_per_second': 6.756, 'total_flos': 2412052070400000.0, 'train_loss': 1.423659377639358, 'epoch': 25.0})

In [17]:
# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset['test'].select(range(10)))
# Evaluate after fine-tuning
post_eval_results = trainer.evaluate(encoded_dataset['test'])

# Print the evaluation losses before and after fine-tuning
print('Evaluation Results before fine-tuning :', pre_eval_results['eval_loss'])
print('Evaluation Results after fine-tuning  :', post_eval_results['eval_loss'])

# Get predictions for validation set before fine tuning for 10 samples
post_val_predictions = trainer.predict(encoded_dataset['test'].select(range(10)))

# Zip the pre and post tuning predictions
predictions = zip(pre_val_predictions.predictions, post_val_predictions.predictions)

Evaluation Results before fine-tuning : 9.061310768127441
Evaluation Results after fine-tuning  : 1.5355618000030518


## Results

In [22]:
for idx, (pre, post) in enumerate(predictions):
    pre_pred = tokenizer.decode(np.argmax(pre, axis=-1), skip_special_tokens=True)
    post_pred = tokenizer.decode(np.argmax(post, axis=-1), skip_special_tokens=True)
    ground_truth = encoded_dataset['test'][idx]["page"]
    
    print(f'Ground truth {idx} \n' + ground_truth + '\n')
    print('Pre-prediction \n' + "".join(pre_pred) + '\n')
    print('Post-prediction \n'+ "".join(post_pred) + '\n')
    print('----------------------------------------------------------------------------------------------------------------------\n')

Ground truth 0 
DAREPÚBLICA—ISÉRIE-A 8161 d) Empresa de manutenção de ascensores (EMA) a entidadequeefectuaeéresponsávelpelamanu- tenção das instalações, cujo estatuto constitui o anexo I a este diploma e que dele faz parte integrante; e) Entidade inspectora (EI) a empresa habilitada a efectuar inspecções a instalações, bem como a realizar inquéritos, peritagens, relatórios e pareceres, cujo estatuto constitui o anexo IV a este diploma e que dele faz parte integrante

Pre-prediction 
 —EPÚBLICA—ISÉRIE-A 8165 8) Apresresa de manutenção de ascensores,M) 1)idade gest souncuar)cnabilvel posutenêsehaã,as Ealações o porjo ascatuto dui- proprietexo IV aoja, que se faz parte integrante; e) Emidade queore deEMAI), eopresa dejailitar;ofetuar;petção�es peroalações, om

Post-prediction 
 —EPÚBLICA—ISÉRIE-A 8165 8) Apresresa de manutenção de ascensores,M) 1)idade gest souncuar)cnabilvel posutenêsehaã,as Ealações o porjo ascatuto dui- proprietexo IV aoja, que se faz parte integrante; e) Emidade queo

# GPT2 IN PORTUGUES
pierreguillou/gpt2-small-portuguese

In [11]:
# https://huggingface.co/pierreguillou/gpt2-small-portuguese

In [13]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Get sequence length max of 1024
tokenizer.model_max_length=1024

/home/peterdays/Documents/personal/Volupal/ElevaQ/env/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1581: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/home/peterdays/Documents/personal/Volupal/ElevaQ/env/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Encoding

In [14]:
import numpy as np
import tempfile
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('pierreguillou/gpt2-small-portuguese')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('pierreguillou/gpt2-small-portuguese')

In [15]:
# Encode the dataset
# https://huggingface.co/docs/transformers/en/pad_truncation
def encode(examples):
    encoded = tokenizer(examples['page'],
                        truncation=True,
                        padding='max_length',
                        max_length=1024
                       )
    encoded['labels'] = encoded['input_ids'][:]

    return encoded

encoded_dataset = dataset.map(encode, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:00<00:00, 1243.54 examples/s]


## Training

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=tempfile.mkdtemp(),   # output directory
    num_train_epochs=5,             # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,                # directory for storing logs
    fp16=True,                       # use floating point 16 bit precision for training
    # eval_accumulation_steps=250  # accumulate the tensors to cpu after 1k steps
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

## Evaluation

In [17]:
# Evaluate before fine-tuning
pre_eval_results = trainer.evaluate(encoded_dataset['test'])

In [18]:
# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset['test'].select(range(10)))

## Fine tuning

In [19]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,1.756600
1000,0.205800
1500,0.180700
2000,0.158000
2500,0.155000
3000,0.151500
3500,0.129100
4000,0.122200
4500,0.117600
5000,0.105300


TrainOutput(global_step=7180, training_loss=0.2449612054320099, metrics={'train_runtime': 1700.71, 'train_samples_per_second': 4.222, 'train_steps_per_second': 4.222, 'total_flos': 3752153579520000.0, 'train_loss': 0.2449612054320099, 'epoch': 5.0})

In [20]:
trainer.save_model("gpt2_portuguese")

In [23]:
pre_eval_results

{'eval_loss': 14.016154289245605,
 'eval_runtime': 27.6294,
 'eval_samples_per_second': 15.056,
 'eval_steps_per_second': 15.056}

In [22]:
post_eval_results

{'eval_loss': 0.14035974442958832,
 'eval_runtime': 27.5374,
 'eval_samples_per_second': 15.107,
 'eval_steps_per_second': 15.107,
 'epoch': 5.0}

In [25]:
type(pre_val_predictions)

transformers.trainer_utils.PredictionOutput

In [21]:
# Evaluate after fine-tuning
post_eval_results = trainer.evaluate(encoded_dataset['test'])

# Print the evaluation losses before and after fine-tuning
print('Evaluation Results before fine-tuning :', pre_eval_results['eval_loss'])
print('Evaluation Results after fine-tuning  :', post_eval_results['eval_loss'])

# Get predictions for validation set before fine tuning for 10 samples
post_val_predictions = trainer.predict(encoded_dataset['test'].select(range(10)))

# Zip the pre and post tuning predictions
predictions = zip(pre_val_predictions.predictions, post_val_predictions.predictions)

Evaluation Results before fine-tuning : 14.016154289245605
Evaluation Results after fine-tuning  : 0.14035974442958832


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.73 GiB. GPU 0 has a total capacty of 5.79 GiB of which 1.72 GiB is free. Including non-PyTorch memory, this process has 4.06 GiB memory in use. Of the allocated memory 3.16 GiB is allocated by PyTorch, and 807.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Results

In [17]:
for idx, (pre, post) in enumerate(predictions):
    pre_pred = tokenizer.decode(np.argmax(pre, axis=-1), skip_special_tokens=True)
    post_pred = tokenizer.decode(np.argmax(post, axis=-1), skip_special_tokens=True)
    ground_truth = encoded_dataset['test'][idx]["page"]

    print(f'Ground truth {idx} \n' + ground_truth + '\n')
    print('Pre-prediction \n' + "".join(pre_pred) + '\n')
    print('Post-prediction \n'+ "".join(post_pred) + '\n')
    print('----------------------------------------------------------------------------------------------------------------------\n')

Ground truth 0 
Diário da República, 1.ª série — N.º 120 — 24 de Junho de 2008 Por ocasião dessas visitas, o organismo notificado poderá, se necessário, efectuar ou mandar efectuar ensaios destinados a verificar o bom funcionamento do sistema de qualidade. Fornecerá ao fabricante um relatório de visita e, caso tenha sido feito um ensaio, um relatório de ensaio

Pre-prediction 
 da República, 1.ª série — N.º 112 — 24 de Junho de 2008 b motivos da emissões, as organismo notificado deve efetuar nos necessário, efetuar, mandar efectuar, para a verificar a bom funcionamento do asc da qualidade, Taisornecerá, organismo, relatório da avaliação e um se lhe sido emitido um ensaio, um relatório da ensaio

Post-prediction 
 da República, 1.ª série — N.º 112 — 24 de Junho de 2008 b motivos da emissões, as organismo notificado deve efetuar nos necessário, efetuar, mandar efectuar, para a verificar a bom funcionamento do asc da qualidade, Taisornecerá, organismo, relatório da avaliação e um se lhe s